# Timing analysis

In [1]:
import datetime
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

In [2]:
os.chdir("..")
utils_path = "utils/"
if not utils_path in sys.path:
    sys.path.insert(0, utils_path)
from helper import *

## Data loading

In [3]:
def duration_to_timedelta(duration):
    duration_comps = duration.split(":")
    if len(duration_comps) > 2:
        hours, minutes, seconds = duration_comps[-3], duration_comps[-2], duration_comps[-1]
    else:
        hours, (minutes, seconds) = (0, duration_comps)
    return pd.Timedelta(hours=float(hours), minutes=float(minutes), seconds=float(seconds))

In [4]:
def read_timing_df(src):
    return pd.read_csv(src, names=["timestamp", "run", "duration"], converters={
        "timestamp": lambda ts: pd.to_datetime(ts, unit="s"),
        "duration": duration_to_timedelta
    })

In [5]:
df_baseline = read_timing_df("ues/workloads/timing-implicit.csv")
df_baseline["workload"] = "baseline"

df_ues = read_timing_df("ues/workloads/timing-explicit.csv")
df_ues["workload"] = "ues"

df_bao = read_timing_df("pg-bao/workloads/timing-full-train.csv")
df_bao["workload"] = "bao"

df_bao_nc = read_timing_df("pg-bao/workloads/timing-full-train-no-cache.csv")
df_bao_nc["workload"] = "bao_nocache"

df_bao_all = pd.concat([df_bao, df_bao_nc]).reset_index(drop=True)
df_bao_all["duration_s"] = df_bao_all.duration.apply(lambda d: d.seconds)
df_bao_all["duration_min"] = df_bao_all.duration_s / 60

df_all = pd.concat([df_baseline, df_ues, df_bao, df_bao_nc]).reset_index(drop=True)
df_all["duration_s"] = df_all.duration.apply(lambda d: d.seconds)
df_all["duration_min"] = df_all.duration_s / 60

## "Evaluation"

**Careful!** The absolute values of the different workload durations can not be compared! Each workload executes differently - a BAO workload consist of three iterations over the input queries, whereas baseline/UES only iterate once over the input queries.

In [6]:
df_all.groupby("workload")["duration_min"].describe()[["mean", "std", "min", "max"]]

,mean,std,min,max
workload,,,,
bao,39.566667,2.386740,36.800000,43.350000
bao_nocache,40.306667,1.634889,37.783333,41.650000
baseline,8.386667,0.730810,7.866667,9.666667
ues,5.006667,0.657816,4.700000,6.183333


Runtimes of the different BAO setups:

In [7]:
for workload, durations in df_bao_all.groupby("workload")["duration_min"]:
    print("===", workload)
    print(sorted(durations.unique()))

=== bao
[36.8, 38.666666666666664, 39.483333333333334, 39.53333333333333, 43.35]
=== bao_nocache
[37.78333333333333, 39.53333333333333, 41.2, 41.36666666666667, 41.65]
